#Контест 2

##Baseline
###Предобработка

In [5]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
%matplotlib inline

In [6]:
features = pandas.read_csv('data/features.csv', index_col='match_id')

In [7]:
matches = features.copy()
del matches['duration']
del matches['radiant_win']
del matches['tower_status_dire']
del matches['tower_status_radiant']
del matches['barracks_status_radiant']
del matches['barracks_status_dire']

In [8]:
matches = matches.fillna(0)

In [12]:
categorical_columns = ['lobby_type']
for i in range(1, 6):
    categorical_columns.append('r'+str(i)+'_hero')
    categorical_columns.append('r'+str(i)+'_hero')
other_cols = [col for col in matches.columns if not (col in categorical_columns)]

In [9]:
transformer = DictVectorizer(sparse=False)
matches_arr = transformer.fit_transform(matches[categorical_columns].astype(str).T.to_dict().values())

In [30]:
train = np.hstack((matches_arr, matches[other_cols]))

In [31]:
train_size = 70000
test = train[train_size:, :]
train = train[:train_size, :]
y_train = np.array(features['radiant_win'])
y_test = y_train[train_size:]
y_train = y_train[:train_size]

In [16]:
def preprocess(data, cat_cols, other_cols, train_size=None):
    if train_size is None:
        train_size = int(2 * data.shape[0] / 3)
    y_train = np.array(data['radiant_win'])
    y_test = y_train[train_size:]
    y_train = y_train[:train_size]
    del data['duration']
    del data['radiant_win']
    del data['tower_status_dire']
    del data['tower_status_radiant']
    del data['barracks_status_radiant']
    del data['barracks_status_dire']
    transformer = DictVectorizer(sparse=False)
    data_arr = transformer.fit_transform(data[cat_cols].astype(str).T.to_dict().values())
    data = np.hstack((data_arr, data[other_cols]))
    test = data[train_size:, :]
    train = data[:train_size, :]
    return train, y_train, test, y_test

###xgboost

In [17]:
import xgboost as xgb

In [18]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test, label=y_test)

NameError: name 'train' is not defined

In [19]:
param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist  = [(dtest,'eval'), (dtrain,'train')]

NameError: name 'dtest' is not defined

In [50]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [46]:
y_pred = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)

In [55]:
roc_auc_score(y_test, y_pred)

0.68126207054796417

##Эксперименты

###Тип Лобби

In [58]:
features['lobby_type'].value_counts()

1    55962
7    28550
0    12718
dtype: int64

In [95]:
features_1 = features[features['lobby_type'] != 7]
features_1 = features_1.fillna(0)

In [96]:
train, y_train, test, y_test = preprocess(features_1, categorical_columns, other_cols)

In [127]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test, label=y_test)
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [128]:
y_pred = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
roc_auc_score(y_test, y_pred)

0.67780254453946687

На типе лобби 7 плохое качество

##Удалим лишние признаки

In [141]:
categorical_columns = ['lobby_type']
for i in range(1, 6):
    categorical_columns.append('r'+str(i)+'_hero')
    categorical_columns.append('r'+str(i)+'_hero')
other_cols = [col for col in matches.columns if not (col in categorical_columns)
              and col != 'start_time' and 'col != lobby_type']

In [142]:
train, y_train, test, y_test = preprocess(features_1.fillna(0), categorical_columns, other_cols)

In [143]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test, label=y_test)
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [144]:
y_pred = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
roc_auc_score(y_test, y_pred)

0.68929945323680353

###Герои

In [9]:
heroes = []
for i in range(1, 6):
    heroes += list(features['r'+str(i)+'_hero'].values)
    heroes += list(features['r'+str(i)+'_hero'].values)
heroes = list(set(heroes))

####Простая сумма характеристик

In [20]:
matches = features.copy()
matches = matches.fillna(0)

In [21]:
fields = ['level', 'gold', 'xp', 'lh', 'kills', 'deaths', 'items']
teams = ['r', 'd']
players = range(1, 6)
for field in fields:
    for team in teams:
        name = team + '_' + field
        matches[name] = 0
        for player in players:
            matches[name] += matches[team + str(player) + '_' + field] 
            del matches[team + str(player) + '_' + field] 

In [22]:
bad_columns = ['duration', 'radiant_win', 'tower_status_dire', 'tower_status_radiant','barracks_status_radiant',
               'barracks_status_dire']
categorical_columns = ['lobby_type']
for i in range(1, 6):
    categorical_columns.append('r'+str(i)+'_hero')
    categorical_columns.append('d'+str(i)+'_hero')
other_cols = [col for col in matches.columns if not (col in categorical_columns)
             and not (col in bad_columns)]
#               and col != 'start_time']

In [23]:
train, y_train, test, y_test = preprocess(matches, categorical_columns, other_cols)

In [24]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test, label=y_test)
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [25]:
y_pred = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
roc_auc_score(y_test, y_pred)

0.70705343471168769

####Взвешенная сумма характеристик
Попробуем для каждого героя посчитать корреляцию его показателей с результатом

In [223]:
matches = features.copy()
matches = matches.fillna(0)

In [224]:
fields = ['hero', 'level', 'gold', 'xp', 'lh', 'kills', 'deaths', 'items']
teams = ['r', 'd']
players = range(1, 6)
field_lst = []
for field in fields:
    for team in teams:
        for player in players:
            field_lst.append(team+str(player)+'_'+field)
field_lst += ['radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire', 
              'barracks_status_radiant', 'barracks_status_dire']

In [225]:
matches = matches[field_lst]

In [237]:
matches_hero = pandas.concat([matches[matches['r1_hero'] == 1]] + [matches[matches['r2_hero'] == 1]])

In [351]:
hero = 93
bad_columns = ['radiant_win']
fields = ['level', 'gold', 'xp', 'lh', 'kills', 'deaths', 'items']
teams = ['r', 'd']
players = range(1, 6)
frames = []
for team in teams:
    for player in players:
        frame = matches[matches[team + str(player) + '_hero'] == hero]
        for field in fields:
            player_field = team + str(player) + '_' + field
            frame[field] = frame[player_field]
        if team == 'd':
            frame[bad_columns] = frame[bad_columns] * (-1) + 1
        frames.append(frame[fields + bad_columns])
    
matches_hero = pandas.concat(frames)

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/IPython/kernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/frame.py:2148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [352]:
matches_hero[matches_hero['radiant_win'] == 1].gold.mean()

1441.8954028925621

In [335]:
matches_arr = np.array(matches_hero)

In [336]:
scaler = StandardScaler()
matches_arr = scaler.fit_transform(matches_arr.astype(float))

In [337]:
matches_arr.shape

(10783, 7)

In [338]:
train_size = int(matches_arr.shape[0] * 2 /3)
train = matches_arr[:train_size, :]
test = matches_arr[train_size:, :]
y_train = matches_hero['radiant_win']
y_test = y_train[train_size:]
y_train = y_train[:train_size]

In [339]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test, label=y_test)
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [340]:
y_pred = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
roc_auc_score(y_test, y_pred)

0.58706529579066413

In [341]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(train, y_train)
y_pred = logreg.predict_proba(test)[:, 1]

In [342]:
roc_auc_score(y_test, y_pred)

0.61268619982893024

####Выбранные герои

In [26]:
matches = features.copy()
matches = matches.fillna(0)

In [27]:
hero_fields = ['r'+str(i)+'_hero' for i in range(1, 6)] + ['d'+str(i)+'_hero' for i in range(1, 6)]
matches_hero = matches[hero_fields + ['radiant_win']]

In [28]:
N = 113
data = matches
X_pick = np.zeros((data.shape[0], N))
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [29]:
train_size = int(matches.shape[0] * 2 / 3)
train = X_pick[:train_size, :]
test = X_pick[train_size:, :]
y_train = matches['radiant_win']
y_test = y_train[train_size:]
y_train = y_train[:train_size]

Логистическая Регрессия

In [30]:
logreg = LogisticRegression()
logreg.fit(train, y_train)
logreg_preds = logreg.predict_proba(test)[:, 1]
roc_auc_score(y_test, logreg_preds)

0.60137657794217358

In [31]:
train_logreg_preds = logreg.predict_proba(train)[:, 1]

In [32]:
full_logreg_preds = np.hstack((train_logreg_preds, logreg_preds))

Бустинг

In [10]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test, label=y_test)
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [11]:
boost_preds = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
roc_auc_score(y_test, boost_preds)

0.56676921118967205

Случайный Лес

In [15]:
forest = RandomForestClassifier(n_estimators=100)
forest.fit(train, y_train)
forest_preds = forest.predict_proba(test)[:, 1]
roc_auc_score(y_test, forest_preds)

0.57763439374103887

SVM

In [ ]:
svm = SVC()
svm.fit(train, y_train)
svm_preds = svm.predict_proba(test)[:, 1]
roc_auc_score(y_test, forest_preds)

####Нейронная сеть

In [63]:
from pybrain.datasets import SupervisedDataSet
from pybrain.utilities import percentError
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.structure.modules import SoftmaxLayer

In [135]:
net = buildNetwork(113, 400, 2, outclass=SoftmaxLayer)

In [136]:
nn_y_train = np.hstack((y_train[:, None], np.logical_not(y_train)[:, None]))
nn_y_test = np.hstack((y_test[:, None], np.logical_not(y_test)[:, None]))

In [137]:
ds = SupervisedDataSet(train.shape[1], 1)
ds.setField('input', train)
ds.setField('target', nn_y_train)

In [138]:
trainer = BackpropTrainer(net, dataset=ds)

In [139]:
trainer.train()

0.17627569609510868

In [143]:
net_preds = []
for example in test[:, :]:
    net_preds.append(net.activate(example)[0])
net_preds = np.array(net_preds)

In [144]:
print(roc_auc_score(y_test, net_preds))

0.561290518885


In [142]:
net.activate(test[5, :])

array([ 0.03385721,  0.96614279])

###Аггрегация информации о героях

In [33]:
matches = features.copy()
matches = matches.fillna(0)
matches['log_heroes'] = full_logreg_preds

In [34]:
fields = ['level', 'gold', 'xp', 'lh', 'kills', 'deaths', 'items']
teams = ['r', 'd']
players = range(1, 6)
for field in fields:
    for team in teams:
        name = team + '_' + field
        matches[name] = 0
        for player in players:
            matches[name] += matches[team + str(player) + '_' + field] 
            del matches[team + str(player) + '_' + field] 

In [35]:
bad_columns = ['start_time', 'duration', 'radiant_win', 'tower_status_dire', 'tower_status_radiant',
               'barracks_status_radiant', 'barracks_status_dire', 'first_blood_player1', 'first_blood_player2']
categorical_columns = ['lobby_type']
for i in range(1, 6):
    bad_columns.append('r'+str(i)+'_hero')
    bad_columns.append('d'+str(i)+'_hero')
#     bad_columns.append()
other_cols = [col for col in matches.columns if not (col in categorical_columns)
             and not (col in bad_columns)]

In [36]:
train, y_train, test, y_test = preprocess(matches, categorical_columns, other_cols)

In [39]:
train.shape

(64820, 36)

In [37]:
dtrain = xgb.DMatrix(train, label=y_train)
dtest = xgb.DMatrix(test, label=y_test)
param = {'bst:max_depth':3, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
# evallist  = [(dtest,'eval'), (dtrain,'train')]
evallist  = [(dtrain,'train')]
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [38]:
boost_preds = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
roc_auc_score(y_test, boost_preds)

0.72304277696606256

In [230]:
categorical_columns

['lobby_type']

###Случайный лес

In [231]:
from sklearn.ensemble import RandomForestClassifier

In [232]:
forest = RandomForestClassifier(n_estimators=100)

In [233]:
forest.fit(train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [234]:
forest_preds = forest.predict_proba(test)[:, 1]

In [235]:
roc_auc_score(y_test, forest_preds)

0.72392289135457544

In [216]:
for alpha in np.linspace(0,1,11):
    print('Alpha', alpha, ':')
    preds = alpha * forest_preds + (1 - alpha) * boost_preds
    print(roc_auc_score(y_test, preds))
    print()

Alpha 0.0 :
0.723042776966

Alpha 0.1 :
0.72505828652

Alpha 0.2 :
0.726769425343

Alpha 0.3 :
0.72813551037

Alpha 0.4 :
0.729110148665

Alpha 0.5 :
0.729646675486

Alpha 0.6 :
0.729691207881

Alpha 0.7 :
0.729209325839

Alpha 0.8 :
0.728175578997

Alpha 0.9 :
0.72658799388

Alpha 1.0 :
0.724396835471



###Сясь

In [237]:
train.shape

(64820, 36)

In [238]:
from sklearn.svm import SVC

In [ ]:
svm = SVC()
svm.fit(train, y_train)

In [ ]:
svm_preds = svm.predict_proba(test)
roc_auc_score(y_test, svm_preds)